Listing all the assement and their url

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
data = []
# Target URL 
for i in range(0, 13):
    url = "https://www.shl.com/solutions/products/product-catalog/?start="+str(i*12)+"&type=2&type=2"

# Send request to SHL website
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
    # Parse HTML content
        soup = BeautifulSoup(response.text, "html.parser")

    # Find the table (Update selector if needed)
        table = soup.find_all("table")
        if( len(table) == 1):
            table = table[0]
        else:
            table = table[1]
    

    # Loop through table rows
        for row in table.find_all("tr")[1:]:  # Skip header row
            cols = row.find_all("td")
            if len(cols) == 4:
                data.append({
                "Assessment Name": cols[0].text.strip(),
                "Remote_Testing": 1 if cols[1].find("span") else 0,
                "adaptive": 1 if cols[2].find("span") else 0,
                    "URL": "https://www.shl.com/"+cols[0].find("a")["href"] if cols[0].find("a") else "N/A"
                })  # Example: Loop through first 5 pages

len(data)


156

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
# data = []
# Target URL
for i in range(0, 32):
    url = "https://www.shl.com/solutions/products/product-catalog/?start=" + str(i*12)+"&type=1&type=1"

# Send request to SHL website
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse HTML content
        soup = BeautifulSoup(response.text, "html.parser")

    # Find the table (Update selector if needed)
        table = soup.find_all("table")
        if (len(table) == 1):
            table = table[0]
        else:
            table = table[1]

    # Loop through table rows
        for row in table.find_all("tr")[1:]:  # Skip header row
            cols = row.find_all("td")
            if len(cols) == 4:
                data.append({
                    "Assessment Name": cols[0].text.strip(),
                    "Remote_Testing": 1 if cols[1].find("span") else 0,
                    "adaptive": 1 if cols[2].find("span") else 0,
                    "URL": "https://www.shl.com/"+cols[0].find("a")["href"] if cols[0].find("a") else "N/A"
                })  # Example: Loop through first 5 pages
len(data)

533

In [20]:
combined_df = pd.DataFrame(data)
combined_df.to_csv(csv_file, index=False)
print("Scraping successful! Data appended to shl_assessments.csv.")
print(combined_df.shape)

Scraping successful! Data appended to shl_assessments.csv.
(533, 4)


In [ ]:
print(new_df.shape)


(156, 4)
(302, 4)


Scraping successful! Data appended to shl_assessments.csv.
(458, 5)


In [21]:
import pandas as pd
df=pd.read_csv('shl_assessments.csv')
df.head()

,Assessment Name,Remote_Testing,adaptive,URL
0,Global Skills Development Report,1,0,https://www.shl.com//solutions/products/produc...
1,.NET Framework 4.5,1,1,https://www.shl.com//solutions/products/produc...
2,.NET MVC (New),1,0,https://www.shl.com//solutions/products/produc...
3,.NET MVVM (New),1,0,https://www.shl.com//solutions/products/produc...
4,.NET WCF (New),1,0,https://www.shl.com//solutions/products/produc...


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Load URLs from CSV
df = pd.read_csv("shl_assessments.csv")  # Replace with actual file
urls = df["URL"].tolist()  # Assuming column name is "URL"

data = []

for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract description
        desc_section = soup.find("h4", text="Description")
        description = desc_section.find_next(
            "p").text.strip() if desc_section else "N/A"

        # Extract job levels
        job_section = soup.find("h4", text="Job levels")
        job_levels = job_section.find_next(
            "p").text.strip() if job_section else "N/A"


        # Store results
        data.append({"URL": url, "Description": description,
                    "Job Levels": job_levels})

# Save to CSV
output_df = pd.DataFrame(data)
output_df.to_csv("scraped_data.csv", index=False)
print("Scraping complete! Data saved to scraped_data.csv")

C:\Users\arpit\AppData\Local\Temp\ipykernel_21544\3313022944.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  desc_section = soup.find("h4", text="Description")
C:\Users\arpit\AppData\Local\Temp\ipykernel_21544\3313022944.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  job_section = soup.find("h4", text="Job levels")


Scraping complete! Data saved to scraped_data.csv


In [2]:
import requests
from bs4 import BeautifulSoup
import re

csv_file_path = "shl_assessments.csv"

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Create an empty list to store assessment lengths
assessment_lengths = []

for url in df['URL']:  # Assuming the column name is 'URL'
    # Debug: Print the URL being fetched
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Debug: Print all h4 tags
        all_h4s = [h4.text for h4 in soup.find_all("h4")]

    # Attempt to find 'Assessment length'
        assessment_length_header = soup.find(
        "h4", string=lambda text: text and "Assessment length" in text)

        if assessment_length_header:
            length_text = assessment_length_header.find_next_sibling(
            "p").text.strip()
            match = re.search(r"\d+", length_text)
            assessment_length = int(match.group()) if match else None
            print("Assessment Length (minutes):", assessment_length)
            assessment_lengths.append(assessment_length)
        else:
            assessment_lengths.append(None)
            print("❌ 'Assessment length' not found in h4 tags.")
    else:
        print("❌ Failed to fetch the page. Status code:", response.status_code)
df['Assessment Length (min)'] = assessment_lengths

# Save the updated DataFrame back to a CSV file
df.to_csv("updated_file.csv", index=False)

❌ 'Assessment length' not found in h4 tags.
Assessment Length (minutes): 30
Assessment Length (minutes): 17
Assessment Length (minutes): 5
Assessment Length (minutes): 11
Assessment Length (minutes): 9
Assessment Length (minutes): 5
Assessment Length (minutes): 9
Assessment Length (minutes): 8
Assessment Length (minutes): 13
Assessment Length (minutes): 8
Assessment Length (minutes): 10
❌ 'Assessment length' not found in h4 tags.
Assessment Length (minutes): 30
Assessment Length (minutes): 17
Assessment Length (minutes): 5
Assessment Length (minutes): 11
Assessment Length (minutes): 9
Assessment Length (minutes): 5
Assessment Length (minutes): 9
Assessment Length (minutes): 8
Assessment Length (minutes): 13
Assessment Length (minutes): 8
Assessment Length (minutes): 10
❌ 'Assessment length' not found in h4 tags.
Assessment Length (minutes): 30
Assessment Length (minutes): 17
Assessment Length (minutes): 5
Assessment Length (minutes): 11
Assessment Length (minutes): 9
Assessment Length 

In [13]:
import pandas as pd

# Load the three CSV files
scrapped_df = pd.read_csv("scraped_data.csv")
updated_df = pd.read_csv("updated_file.csv")
shl_df = pd.read_csv("shl_assessments.csv")

# Print available column names to debug
print("Scrapped Data Columns:", scrapped_df.columns)
print("Updated File Columns:", updated_df.columns)
print("SHL Data Columns:", shl_df.columns)

# Define common columns for merging (adjust if needed)
common_columns = list(set(scrapped_df.columns) & set(
    updated_df.columns) & set(shl_df.columns))

# Merge dataframes using outer join
merged_df = scrapped_df.merge(updated_df, on=common_columns, how="outer").merge(
    shl_df, on=common_columns, how="outer"
)

# Check if 'Assessment Name' exists before setting index

    # Set index
# merged_df.set_index("URL", inplace=True)
#     # Save to CSV
# merged_df.to_csv("final_combined.csv")
# print("✅ Merged CSV saved successfully after dropping NaN values.")

Scrapped Data Columns: Index(['URL', 'Description', 'Job Levels'], dtype='object')
Updated File Columns: Index(['Assessment Name', 'Remote_Testing', 'adaptive', 'URL',
       'Assessment Length (min)'],
      dtype='object')
SHL Data Columns: Index(['Assessment Name', 'Remote_Testing', 'adaptive', 'URL'], dtype='object')


In [17]:
print(merged_df.columns)


Index(['URL', 'Description', 'Job Levels', 'Assessment Name_x',
       'Remote_Testing_x', 'adaptive_x', 'Assessment Length (min)',
       'Assessment Name_y', 'Remote_Testing_y', 'adaptive_y'],
      dtype='object')


In [18]:
columns_to_drop = ["Assessment Name_y", "Remote_Testing_y", "adaptive_y"]
merged_df.drop(columns=columns_to_drop, inplace=True,
               errors="ignore")  # ignore if column is missing

# Rename 'Assessment Name_x' to 'Assessment Name'
merged_df.rename(
    columns={"Assessment Name_x": "Assessment Name"}, inplace=True)

In [22]:
merged_df.rename(
    columns={"Remote_Testing_x": "Remote_Testing", 
             "adaptive_x":"adaptive"}, inplace=True)

In [23]:
merged_df.to_csv("final_cleaned.csv", index=False)

print("✅ Cleaned CSV saved as 'final_cleaned.csv'.")

✅ Cleaned CSV saved as 'final_cleaned.csv'.
